# Question A. Data Analysis

In [4]:
import sqlite3
import os
conn = sqlite3.connect('webvisit.sqlite')
c = conn.cursor()

In [5]:
try:
    c.execute('CREATE TABLE webvisit_db (userid INTEGER, pageid INTEGER, visitime TEXT)')
except:
    pass

In [6]:
import csv
with open('junior-data-test.csv', 'rt') as csvfile:
    dr = csv.reader(csvfile, delimiter=',', quotechar='|')
    for t in dr:
        c.execute('INSERT INTO webvisit_db VALUES (?,?,?)', t)
conn.commit()
conn.close()

In [7]:
# Q.A How many visits are in the data set?
conn = sqlite3.connect('webvisit.sqlite')
c = conn.cursor()
c.execute('SELECT COUNT(*) FROM webvisit_db')
result = c.fetchall()
conn.close()
print('Visits in the data set:')
print(result[0][0])

Visits in the data set:
357912


In [8]:
# Q.A How many distinct users are in the data set?
conn = sqlite3.connect('webvisit.sqlite')
c = conn.cursor()
c.execute('SELECT COUNT(DISTINCT userid) FROM webvisit_db')
result = c.fetchall()
conn.close()
print('Distinct users are in the data set:')
print(result[0][0])

Distinct users are in the data set:
64265


In [9]:
# Q.A How many distinct pages are in the data set?
conn = sqlite3.connect('webvisit.sqlite')
c = conn.cursor()
c.execute('SELECT COUNT(DISTINCT pageid) FROM webvisit_db')
result = c.fetchall()
conn.close()
print('Distinct pages in the data set:')
print(result[0][0])

Distinct pages in the data set:
15163


In [10]:
# Q.A Which hour gives the smallest number of visits?
conn = sqlite3.connect('webvisit.sqlite')
c = conn.cursor()
c.execute("""SELECT a.hour, MIN(a.hour_cnt) AS min"""\
          """ FROM (SELECT strftime('%H', visitime) AS hour, COUNT(*) AS hour_cnt FROM webvisit_db GROUP BY 1) a""")
result = c.fetchall()
conn.close()
print('Hour gives the smallest number of visits:')
print('%s:00:00 - %s:59:59' % (result[0][0], result[0][0]))

Hour gives the smallest number of visits:
04:00:00 - 04:59:59


In [11]:
# Q.A Which hour gives the largest number of visits?
conn = sqlite3.connect('webvisit.sqlite')
c = conn.cursor()
c.execute("""SELECT a.hour, MAX(a.hour_cnt)"""\
          """ FROM (SELECT strftime('%H', visitime) AS hour, COUNT(*) AS hour_cnt FROM webvisit_db GROUP BY 1) a""")
result = c.fetchall()
conn.close()
print('Hour gives the smallest number of visits:')
print('%s:00:00 - %s:59:59' % (result[0][0], result[0][0]))

Hour gives the smallest number of visits:
12:00:00 - 12:59:59


In [12]:
# Q.A Which page has the largest number of visits in the data set? What is the corresponding number of visits?
conn = sqlite3.connect('webvisit.sqlite')
c = conn.cursor()
c.execute('SELECT pageid, MAX(DISTINCT(pageid)) FROM webvisit_db')
result = c.fetchall()
conn.close()
print('Largest number of visits in the data set and corresponding number of visits:')
print(result[0])

Largest number of visits in the data set and corresponding number of visits:
(6819, 15163)


# Question B. Data Visualization

I am not familiar with visualization within a web browser. I know **D3.js** which is JavaScript might be proper in doing this task. Here is [link](https://d3js.org/#introduction). 

# Question C. Data Science - Tagging prediction
In fact, I used the traditional **logistic regression model** for this new classification since the task is quite simple. After loading the data from the csv file, the first important step is to clean the text data by stripping it of all unwanted characters, for example HTML markup or punctuation. Then I process the text into tokens with **jieba**. After that, I vectorizer the text using **HashingVectorizer** from *scikit-learn*. Having set up all the complementary functions, I can start to train the logistic regression model with stochastic gradient descent.

Answers to the questions are at the end after code. The accuracy is around 99% in my test.

In [13]:
import pandas as pd
import numpy as np
df = pd.read_csv('./offsite-test-material/offsite-tagging-training-set.csv')

In [14]:
# See some examples of data
df.tail()

,id,tags,text
3889,26483,足球,【歐國盃．深宵福利】空手道索太食譜加愛心 成就金靴「羅拔仔」 有誰自認入得廚房，出得Gym房...
3890,3231,足球,阿士東維拉驚險過關 英足盃兩球淨勝乙組仔 今季走勢低迷的阿士東維拉，在英超榜尾苦苦掙扎，轉到...
3891,90791,足球,【港足日與夜．朱兆基】37歲唔認老不服輸 更要喚起南華霸氣 周日（5月14日）於足總盃再撼傑...
3892,56574,梁振英,【特首選戰】梁營啟動競選工程 三女將婉拒入連任辦 羅范工作困身 婉拒「歸隊」\r\r\n\r...
3893,77253,足球,【英超】曼城撼利物浦求反底 效率王辛尼下半季爆發成關鍵 與阿古路一同下半季發力的，不是倒戈利...


In [15]:
# Mapping labels to index
class_mapping = {label: idx for idx, label in enumerate(np.unique(df.tags))}
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df.tags = df.tags.map(class_mapping)
df.tail()

,id,tags,text
3889,26483,2,【歐國盃．深宵福利】空手道索太食譜加愛心 成就金靴「羅拔仔」 有誰自認入得廚房，出得Gym房...
3890,3231,2,阿士東維拉驚險過關 英足盃兩球淨勝乙組仔 今季走勢低迷的阿士東維拉，在英超榜尾苦苦掙扎，轉到...
3891,90791,2,【港足日與夜．朱兆基】37歲唔認老不服輸 更要喚起南華霸氣 周日（5月14日）於足總盃再撼傑...
3892,56574,0,【特首選戰】梁營啟動競選工程 三女將婉拒入連任辦 羅范工作困身 婉拒「歸隊」\r\r\n\r...
3893,77253,2,【英超】曼城撼利物浦求反底 效率王辛尼下半季爆發成關鍵 與阿古路一同下半季發力的，不是倒戈利...


In [16]:
# Cleaning text data to remove HTML markup (</a>), emoticons and so on
import re
# Here I utilized the recommened Chinese word tokenization package jieba to do the word segmentation
import jieba
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text) # remove HTML markup
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text) # find emoticons
    text = re.sub('[\W]+', ' ', text) + ''.join(emoticons).replace('-', '') # add emoticons to the end
    tokenized = [w for w in jieba.lcut(text, cut_all=False)]
    return tokenized

In [17]:
# Testing the cleaning tokenizer
tokenizer("</a>你 :) 今天 :( 很漂亮:-)!")

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/34/h2dy2w_52v791hl1m4wrsft40000gn/T/jieba.cache
Loading model cost 0.989 seconds.
Prefix dict has been built succesfully.


['你', ' ', '今天', ' ', '很漂亮', ' ', ':', ')', ':', '(', ':', ')']

In [18]:
def stream_docs(df):
    for a in range(0, len(df.text)):
        yield df.text[a:a+1].values.tolist()[0], df.tags[a:a+1].values.tolist()[0]
        
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
            return None, None
    return docs, y

In [19]:
# To verify our stream_docs function works correctly
next(stream_docs(df))

('利物浦重賽擊敗乙組仔\u3000英足盃過關 英格蘭足總盃第三圈今晨重賽，貴為英超勁旅的利物浦上場被乙組仔埃克斯特尷尬逼和，多獲一次機會的紅軍不敢再有差池。先有近期回勇的「威爾斯沙維」祖阿倫10分鐘開紀錄，加上兩個小將舒爾奧祖，及祖奧迪西拿下半場各入一球，以3比0擊敗對手，總算在主場挽 <p style="text-align: justify;">英格蘭足總盃第三圈今晨重賽，貴為英超勁旅的利物浦上場被乙組仔埃克斯特尷尬逼和，多獲一次機會的紅軍不敢再有差池。先有近期回勇的「威爾斯沙維」祖阿倫10分鐘開紀錄，加上兩個小將舒爾奧祖，及祖奧迪西拿下半場各入一球，以3比0擊敗對手，總算在主場挽回面子，下一圈對手為韋斯咸。</p> <p style="text-align: justify;">另一場英超球隊對壘，今季異軍突起的李斯特城戰至第三圈就宣告畢業。熱刺憑韓國前鋒孫興<U+615C>上半場遠射破網先開紀錄，換邊後此子助攻予中場查迪尼建功，令球隊以兩球輕取李斯特城，第四圈將面對英甲的高車士打。</p>',
 2)

In [20]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore',
                        n_features=2**21,
                        preprocessor=None,
                        tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(df)

In [21]:
# Start the learning progress. Train the model using totally 3000 samples from the training set
import pyprind
pbar = pyprind.ProgBar(30)
classes = np.array([0, 1, 2])
for _ in range(30):
    X_train, y_train = get_minibatch(doc_stream, size=100)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:33


In [22]:
# Using next 500 samples from the training set to test the model
X_test, y_test = get_minibatch(doc_stream, size=1)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 1.000


In [24]:
# See the result of remaining training data one by one. You can run this multiply times
X_test, y_test = get_minibatch(doc_stream, size=1)
print(X_test, inv_class_mapping[y_test[0]])
print('-'*30)
X = vect.transform(X_test)
label = inv_class_mapping
print('Prediction: %s\nProbability: %.2f%%' % (inv_class_mapping[clf.predict(X)[0]], np.max(clf.predict_proba(X))*100))

['【歐聯】曼城反負摩納哥\u3000防線形同虛設\u3000哥帥首16強止步（有片） 摩納哥收復首回合落後3︰5的劣勢，主場以3︰1挫曼城，總比數6︰6打和憑作客入球優惠晉級8強，這也是曼城主帥哥迪奧拿執教生涯首次於歐聯16強止步。 需要最少入兩球才有機會晉級的摩納哥未有起用法卡奧或摩天奴擔正，一班年輕新星甫開賽便極之進取，8分鐘貝拿度施華左路傳中，由突破越位的基利安洛天射入近3場第4球。左路成為曼城死症，29分鐘班捷文文迪再於類似位置傳中，通天老倌法賓奴泰華利斯撞射入 <p style="text-align: justify;">需要最少入兩球才有機會晉級的摩納哥未有起用法卡奧或摩天奴擔正，一班年輕新星甫開賽便極之進取，8分鐘貝拿度施華左路傳中，由突破越位的基利安洛天射入近3場第4球。左路成為曼城死症，29分鐘班捷文文迪再於類似位置傳中，通天老倌法賓奴泰華利斯撞射入網，令摩納哥在總比數追成5︰5，若保持賽果至完場將晉級。</p>\r\r\n\r\r\n<p style="text-align: justify;"><strong>曼城右閘位成漏洞</strong></p>\r\r\n\r\r\n<p style="text-align: justify;">上半場起腳未嘗中框的曼城下半場略有改善，前鋒阿古路多次近門機會但都未能轉化為入球，至71分鐘終憑利萊辛尼執死雞，再次在總比數領前。可是曼城防守再次走漏人，被摩納哥的泰莫爾巴卡約高接應罰球頂入。曼城換入前鋒伊恩拿祖仍無力再入球，令李斯特城成為8強唯一英格蘭球會。</p> <p style="text-align: justify;"><strong>李斯特城成8強唯一英格蘭代表</strong></p>\r\r\n\r\r\n<p style="text-align: justify;">首次16強出局的哥迪奧拿認為死因是開局太差以及死球是致命傷︰「下半場做得很好，但上半場則忘記了。我們未能踢出應有水平，要好好學習，始終這支球隊欠缺經驗。出局是因為下半場有機會但未能把握。」摩納哥主帥渣甸則認為晉級實至名歸︰「我們防線壓前，前場表現也很好。下半場對手適應了，以兩個前鋒施壓才能減少曼城控球機會。」歐聯8強抽籤將在3月17日（周五）晚上舉行。</p>'] 足球
---------------------------

# See results from testing set

In [25]:
df_test = pd.read_csv('./offsite-test-material/offsite-tagging-test-set.csv')

In [26]:
# See some example of data
df_test.tail()

,id,text
969,93507,【熱刺訪港】普捷天奴成搶手貨 主席利維開腔派定心丸 英超今季群雄割據，摩連奴、干地及哥迪奧拿...
970,93651,【熱刺訪港】孫興<U+615C>林志堅再聚舊 承諾賽後交換球衣 熱刺周五將與傑志於香港大球場...
971,93690,【港足日與夜．王振鵬】膠唔會膠一世 神經刀變神龍（有片） 有些球員出道十多年，一直都被人睇低...
972,93985,【中超】泰維斯抱怨遭中超球員踢傷 澄清無意離開上海申花 受人錢財不一定替人消災，阿根廷前鋒泰...
973,94324,【傑志對熱刺．來稿】睇波睇到開party咁先至過癮 剛過去的周末，香港刮起一股足球熱。先是上...


In [27]:
X = vect.transform(df_test.text)
label = inv_class_mapping
y = []
for i in range(X.shape[0]):
    y.append(label[clf.predict(X[i])[0]])   
print('My predicted tags for the test set:')
print(y)

My predicted tags for the test set:
['足球', '梁振英', '足球', '足球', '梁振英', '梁振英', '梁振英', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '梁振英', '梁振英', '梁振英', '美國大選', '梁振英', '梁振英', '梁振英', '梁振英', '梁振英', '足球', '梁振英', '足球', '梁振英', '足球', '梁振英', '足球', '足球', '足球', '足球', '梁振英', '美國大選', '足球', '美國大選', '足球', '梁振英', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '美國大選', '足球', '足球', '美國大選', '足球', '足球', '足球', '足球', '足球', '足球', '美國大選', '美國大選', '美國大選', '美國大選', '足球', '足球', '美國大選', '足球', '足球', '足球', '足球', '足球', '梁振英', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '美國大選', '梁振英', '美國大選', '美國大選', '美國大選', '足球', '梁振英', '足球', '足球', '足球', '美國大選', '梁振英', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '美國大選', '足球', '足球', '足球', '足球', '梁振英', '足球', '足球', '足球', '梁振英', '美國大選', '梁振英', '足球', '足球', '足球', '梁振英', '梁振英', '美國大選', '美國大選', '美國大選', '美國大選', '足球', '足球', '美國大選', '梁振英', '美國大選', '梁振英', '美國大選', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '美國大選', '美國大選', '美國大選', '足球', '足球',

In [28]:
# Add tags column to dataframe
df_test.insert(loc=1, column='tags', value=y)

In [29]:
pd.DataFrame(df_test).to_csv('./offsite-test-set-with-tags.csv', index=False)

In [30]:
df = pd.read_csv('./offsite-test-set-with-tags.csv')
df.tail()

,id,tags,text
969,93507,足球,【熱刺訪港】普捷天奴成搶手貨 主席利維開腔派定心丸 英超今季群雄割據，摩連奴、干地及哥迪奧拿...
970,93651,足球,【熱刺訪港】孫興<U+615C>林志堅再聚舊 承諾賽後交換球衣 熱刺周五將與傑志於香港大球場...
971,93690,足球,【港足日與夜．王振鵬】膠唔會膠一世 神經刀變神龍（有片） 有些球員出道十多年，一直都被人睇低...
972,93985,足球,【中超】泰維斯抱怨遭中超球員踢傷 澄清無意離開上海申花 受人錢財不一定替人消災，阿根廷前鋒泰...
973,94324,足球,【傑志對熱刺．來稿】睇波睇到開party咁先至過癮 剛過去的周末，香港刮起一股足球熱。先是上...


## Question 1: How well does your model perform
## Answer:
I use 3000 samples from the training set to train my model and 500 samples from the training set to test. I got 98%-100% accuracy when I train it several times. Below is the code for my model.

## Question 2: How did you choose the parameters of the final model
## Answer:
I did not quite get this question. The model was trained by SGD (stochastic gradient descent) and the parameters was were updated through the training process.

## Question 3: On a high level, please explain your final model’s structure, and how it predicts tags from the article text
## Answer:
In fact, I used the traditional **logistic regression model** for this new classification since the task is quite simple. After loading the data from the csv file, the first important step is to clean the text data by stripping it of all unwanted characters, for example HTML markup or punctuation. Then I process the text into tokens with **jieba**. After that, I vectorizer the text using **HashingVectorizer** from *scikit-learn*. Having set up all the complementary functions, I can start to train the logistic regression model with stochastic gradient descent.